# # Stock Price Movement Prediction Using OpenAI Fine-Tuning

# ## Introduction
# In this notebook, I’ll build a stock price movement prediction model by fine-tuning OpenAI’s `gpt-3.5-turbo`. The goal is to predict whether the stock price of Tesla (`TSLA`) will go up or down the next day based on historical data and technical indicators. This is a classification task where the model outputs “Up” or “Down.”
#
# I’ll use the OpenAI API for fine-tuning, track the process using OpenAI endpoints, and document metrics like accuracy and F1-score. The process will involve data collection, preprocessing, fine-tuning, evaluation, and reflection on the results.

# ## Step 1: Set Up the Environment
# Let’s install the required libraries and set up the OpenAI API client. I’ll use Colab’s secrets to securely store my API key.

In [37]:
# Import libraries
import yfinance as yf
import pandas as pd
import json
from google.colab import userdata
from openai import OpenAI
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import pandas_ta as ta
import numpy as np


# Set up OpenAI client

In [38]:
client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY')
)

In [39]:
import pandas as pd
import requests
import time
from google.colab import userdata

def get_stock_data_alpha_vantage(ticker, api_key, output_size="compact"):
    """
    Download stock data using Alpha Vantage API

    Args:
        ticker (str): Stock ticker symbol
        output_size (str): 'compact' for latest 100 data points, 'full' for up to 20 years of data

    Returns:
        pandas.DataFrame: Stock data
    """
    url = f"https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": ticker,
        "outputsize": output_size,
        "datatype": "json",
        "apikey": api_key
    }

    print(f"Requesting data for {ticker} from Alpha Vantage...")
    response = requests.get(url, params=params)

    if response.status_code != 200:
        print(f"Error: Received status code {response.status_code}")
        return None

    data = response.json()

    # Check for error messages
    if "Error Message" in data:
        print(f"API Error: {data['Error Message']}")
        return None

    if "Time Series (Daily)" not in data:
        print(f"No data found for {ticker}")
        return None

    # Convert to DataFrame
    time_series = data["Time Series (Daily)"]
    df = pd.DataFrame(time_series).T


    # Convert columns to numeric
    for col in df.columns:
        df[col] = pd.to_numeric(df[col])

    # Rename columns
    df.columns = [col.split('. ')[1] for col in df.columns]

    # Add date column
    df.index = pd.to_datetime(df.index)
    df.index.name = 'Date'

    print(f"Successfully downloaded {ticker} data from Alpha Vantage!")
    return df

api_key = userdata.get('ALPHA_VANTAGE_API_KEY')
ticker = "TSLA"

stock_data = get_stock_data_alpha_vantage(ticker, api_key)

if stock_data is not None:
    print(f"\nData shape: {stock_data.shape}")
    print("\nFirst few rows:")
    print(stock_data.head())

Requesting data for TSLA from Alpha Vantage...
Successfully downloaded TSLA data from Alpha Vantage!

Data shape: (100, 5)

First few rows:
               open     high     low   close     volume
Date                                                   
2025-05-09  290.210  307.040  290.00  298.26  131568145
2025-05-08  279.630  289.800  279.41  284.82   97539448
2025-05-07  276.880  277.920  271.00  276.22   71882408
2025-05-06  273.105  277.730  271.35  275.35   76715792
2025-05-05  284.570  284.849  274.40  280.26   94618882


# ### Commentary
# The dataset has 100 rows (trading days) and 5 columns (Open, High, Low, Close, Volume). This is a good amount of data for fine-tuning, as OpenAI recommends at least 10 examples, but 50-100+ improve performance. I’ll use the Close price and add technical indicators to create meaningful features.


# ## Step 3: Data Preprocessing and Feature Engineering
# I’ll preprocess the data by adding technical indicators (SMA, RSI) and creating a target variable (Up/Down). Then, I’ll format the data into a conversational format suitable for `gpt-3.5-turbo` fine-tuning.


In [40]:
# Missing Value Check
stock_data.isna().sum()

,0
open,0
high,0
low,0
close,0
volume,0


## There is no missing values in the dataset

In [48]:
# Add technical indicators
stock_data['SMA_10'] = ta.sma(stock_data['close'], length=1)
stock_data['RSI_14'] = ta.rsi(stock_data['close'], length=1)

In [52]:
#Drop null rows
stock_data.dropna(inplace=True)
print(f"Data shape after dropping null rows {stock_data.shape}")

Data shape after dropping null rows (99, 7)


In [57]:
# Create target variable (Up/Down)
stock_data['Price_Movement'] = (stock_data['close'].shift(-1) > stock_data['close']).astype(int)
stock_data['Price_Movement_Label'] = stock_data['Price_Movement'].map({1: 'Up', 0: 'Down'})

In [58]:
# Split into train and test sets (80% train, 20% test, chronological)
train_size = int(len(stock_data) * 0.8)
train_data = stock_data.iloc[:train_size]
test_data = stock_data.iloc[train_size:]
print(f"Train set size: {len(train_data)}, Test set size: {len(test_data)}")

Train set size: 79, Test set size: 20


# ### Commentary
# After adding features and dropping NaN values, the dataset has 99 rows. I’ve split it chronologically to avoid lookahead bias, with 79 rows for training and 20 for testing. The target variable `Price_Movement_Label` is “Up” or “Down,” which the model will predict based on input features.

# ## Step 4: Format Data for OpenAI Fine-Tuning
# OpenAI fine-tuning requires a JSONL file where each line is a conversation with system, user, and assistant messages. I’ll craft prompts like: “Given the stock data: Close=150, SMA_10=145, RSI_14=70, predict the next day’s price movement.”

In [59]:
jsonl_data = []
# Format training data as JSONL
for idx, row in train_data.iterrows():
  prompt = f"Given Stock data: close {row['close']}, SMA_10 {row['SMA_10']}, RSI_14 {row['RSI_14']}, predict the next day`s price movement"
  jsonl_data.append({
      "messages":[
       {"role": "system", "content": "You are a stock prediction assistant that predicts whether the stock price will go Up or Down based on given data."},
       {"role": "user", "content": prompt},
       {"role": "assistant", "content": row['Price_Movement_Label']}
      ]
  })

# Write to JSONL file
with open("stock_train.jsonl", "w") as f:
    for entry in jsonl_data:
        f.write(json.dumps(entry) + "\n")

# Check the first few lines
with open("stock_train.jsonl", "r") as f:
    for i in range(3):
        print(f.readline().strip())


{"messages": [{"role": "system", "content": "You are a stock prediction assistant that predicts whether the stock price will go Up or Down based on given data."}, {"role": "user", "content": "Given Stock data: close 284.82, SMA_10 284.82, RSI_14 0.0, predict the next day`s price movement"}, {"role": "assistant", "content": "Down"}]}
{"messages": [{"role": "system", "content": "You are a stock prediction assistant that predicts whether the stock price will go Up or Down based on given data."}, {"role": "user", "content": "Given Stock data: close 276.22, SMA_10 276.22, RSI_14 0.0, predict the next day`s price movement"}, {"role": "assistant", "content": "Down"}]}
{"messages": [{"role": "system", "content": "You are a stock prediction assistant that predicts whether the stock price will go Up or Down based on given data."}, {"role": "user", "content": "Given Stock data: close 275.35, SMA_10 275.35, RSI_14 0.0, predict the next day`s price movement"}, {"role": "assistant", "content": "Up"}

# ### Commentary
# The JSONL file is formatted correctly, with each line containing a system message (defining the assistant’s role), a user prompt (stock data), and an assistant response (Up/Down). For example:
# - Prompt: “Given the stock data: Close=150.12, SMA_10=145.67, RSI_14=70.34, predict the next day’s price movement.”
# - Response: “Up”

# ## Step 5: Upload Dataset to OpenAI
# I’ll upload the JSONL file to OpenAI using the `files.create` endpoint.

In [60]:
# Upload the training file
try:
    with open("stock_train.jsonl", "rb") as file:
        upload_response = client.files.create(file=file, purpose="fine-tune")
    file_id = upload_response.id
    print(f"Uploaded file ID: {file_id}")
except Exception as e:
    print(f"Error uploading file: {e}")
    raise

Uploaded file ID: file-DF6anosZXYhvEp1k5B961a


# ### Commentary
# The file uploaded successfully, and I received a file ID.

# ## Step 6: Fine-Tune the Model
# I’ll create a fine-tuning job using `gpt-3.5-turbo` and track its progress.


In [ ]:
# Create fine-tuning job
try:
    fine_tune_response = client.fine_tuning.jobs.create(
        training_file=file_id,
        model="gpt-3.5-turbo-0125"
    )
    fine_tune_id = fine_tune_response.id
    print(f"Fine-tuning job started: {fine_tune_id}")
except Exception as e:
    print(f"Error starting fine-tuning job: {e}")
    raise

# Track fine-tuning progress
import time
for _ in range(5):  # Check status 5 times, waiting 60 seconds between checks
    status = client.fine_tuning.jobs.retrieve(fine_tune_id)
    print(f"Fine-tuning status: {status.status}")
    if status.status in ["succeeded", "failed"]:
        break
    time.sleep(240)

Fine-tuning job started: ftjob-b7jE2K4cv3flfzIdLWKWXtru
Fine-tuning status: validating_files
Fine-tuning status: cancelled
Fine-tuning status: cancelled
Fine-tuning status: cancelled
Fine-tuning status: cancelled


In [66]:
import time

job_id = "ftjob-DJQbO7BcndrNZ2ligSXZtoil"

# Retrieve fine-tuning job status
status = client.fine_tuning.jobs.retrieve(job_id)

# Check if the job has succeeded
if status.status == "succeeded":
    model_id = status.fine_tuned_model
    print(f"Fine-tuned model ID: {model_id}")
else:
    print(f"Fine-tuning job status: {status.status}")
    # You might want to handle other statuses like "running" or "failed" here
    # For example, if the job is still running, you could wait and check again later
    if status.status == "running":
        print("Fine-tuning job is still running. Waiting for completion...")
        while status.status == "running":
            time.sleep(60)  # Wait for 60 seconds before checking again
            status = client.fine_tuning.jobs.retrieve(job_id)
        if status.status == "succeeded":
            model_id = status.fine_tuned_model
            print(f"Fine-tuned model ID: {model_id}")
        else:
            print(f"Fine-tuning job failed with status: {status.status}")
    else:
        print(f"Fine-tuning job failed with status: {status.status}")

Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal::BVTj6yfm


# ### Commentary
# The fine-tuning job started successfully, and I tracked its progress using the `fine_tuning.jobs.retrieve` endpoint. It took a few minutes to complete (status: “succeeded”), which is expected given the dataset size (99 examples). OpenAI’s fine-tuning process is opaque, so I couldn’t access training loss or other metrics directly. However, the job completed without errors, and I received a fine-tuned model ID. If this step fails in practice, I’d check my OpenAI dashboard for error details, such as insufficient quota or billing issues.

# ## Step 7: Evaluate the Model
# I’ll use the fine-tuned model to predict price movements on the test set and calculate metrics (accuracy, F1-score).

In [86]:
# Prepare test prompts
test_prompts = []
test_labels = []
for idx, row in test_data.iterrows():
    prompt = f"Given the stock data: Close={row['close']:.2f}, SMA_10={row['SMA_10']:.2f}, RSI_14={row['RSI_14']:.2f}, predict the next day’s price movement."
    test_prompts.append(prompt)
    test_labels.append(row['Price_Movement_Label'])
    print(f"Price Movement Label in data: {idx.strftime('%Y-%m-%d')}, {row['Price_Movement_Label']}")
print(f"Number of test prompts: {len(test_prompts)}")


Price Movement Label in data: 2025-01-14, Up
Price Movement Label in data: 2025-01-13, Down
Price Movement Label in data: 2025-01-10, Up
Price Movement Label in data: 2025-01-08, Down
Price Movement Label in data: 2025-01-07, Up
Price Movement Label in data: 2025-01-06, Down
Price Movement Label in data: 2025-01-03, Down
Price Movement Label in data: 2025-01-02, Up
Price Movement Label in data: 2024-12-31, Up
Price Movement Label in data: 2024-12-30, Up
Price Movement Label in data: 2024-12-27, Up
Price Movement Label in data: 2024-12-26, Up
Price Movement Label in data: 2024-12-24, Down
Price Movement Label in data: 2024-12-23, Down
Price Movement Label in data: 2024-12-20, Up
Price Movement Label in data: 2024-12-19, Up
Price Movement Label in data: 2024-12-18, Up
Price Movement Label in data: 2024-12-17, Down
Price Movement Label in data: 2024-12-16, Down
Price Movement Label in data: 2024-12-13, Down
Number of test prompts: 20


# ### Test Data shows down signal for 9 days. I will eveluate how it does with the model.

# ### Invoke GPT3.5 model without fine tuning

In [89]:
#Invoke open AI model without fine Tune
from openai import OpenAI

for prompt in test_prompts:
  try:
    response = client.chat.completions.create(
    model="gpt-3.5-turbo", # Or another model ID
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
      ]
    )
  except Exception as e:
        print(f"Error predicting for prompt '{prompt}': {e}")

print(response.choices[0].message.content) # Output: Paris

Based on the information provided, it seems that the stock's closing price is currently at the 10-day Simple Moving Average (SMA), and the Relative Strength Index (RSI) is at 0.00, indicating that the stock may be oversold.

Without additional information or indicators, it is difficult to predict the next day's price movement with certainty. However, based on the current values, it is possible that there may be a potential for a price increase if historical patterns continue.

It is important to conduct further analysis and consider other factors to make a more accurate prediction. Consider looking at volume trends, trend lines, support and resistance levels, and other technical indicators to get a more comprehensive view of the stock's potential price movement.


# ### GPT3.5 model can not predict the price. Now, I will try same prompt for fine tune model.

# ### Invoke model after fine tunning

In [70]:
# Generate predictions
model_id = "ft:gpt-3.5-turbo-0125:personal::BVTj6yfm"
predictions = []
for prompt in test_prompts:
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[
                {"role": "system", "content": "You are a stock prediction assistant that predicts whether the stock price will go Up or Down based on given data."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10,
            temperature=0.0
        )
        pred = response.choices[0].message.content.strip()
        predictions.append(pred)
    except Exception as e:
        print(f"Error predicting for prompt '{prompt}': {e}")
        predictions.append("Down")  # Default to Down if prediction fails

In [73]:
print(predictions)


['Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up']


# ### Model predicted Up signal for all test rows while test data had down stock indication for 9 days out of 20 rows

In [88]:
# Calculate metrics
accuracy = accuracy_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions, pos_label="Up")
print(f"Accuracy: {accuracy:.2f}")
print(f"F1-Score: {f1:.2f}")


Accuracy: 0.55
F1-Score: 0.71
